In [1]:
#Importing the necessary packages for the workspace and experiment.
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-130297")
exp = Experiment(workspace=ws, name="AUTO_ML")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-130297
Azure region: southcentralus
Subscription id: 2552278b-2817-43a7-820e-5a5a53ff9e19
Resource group: aml-quickstarts-130297


In [2]:
#Importing the necessary packages for computetarget, amlcompute and computetargetexception.
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cluster_name="AUTOM-ML-cluster"
try:
  compute_target=ComputeTarget(workspace=ws,name=cluster_name)
  print("Found existing compute target")
except ComputeTargetException:
  print('Creating a new compute target')
  compute_config=AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',max_nodes=4)
  compute_target=ComputeTarget.create(ws,cluster_name,compute_config)
  

compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

### YOUR CODE HERE ###

Creating a new compute target
AUTOM-ML-cluster AmlCompute Creating
ds12 AmlCompute Succeeded
sds12 ComputeInstance Succeeded


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_path = "https://sds12.southcentralus.instances.azureml.ms/edit/train.csv.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_path)

In [4]:
from azureml.core import Dataset
full_data=Dataset.Tabular.from_delimited_files(data_path)

In [12]:
import os
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/sds12/code'

In [13]:
import numpy as np
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/AmDeep/Capstone_Project3_AzureML_Nanodegree/main/Data/train.csv")

In [15]:
test = pd.read_csv("https://raw.githubusercontent.com/AmDeep/Capstone_Project3_AzureML_Nanodegree/main/Data/test.csv")

In [16]:
train.describe()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
count,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,...,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000,3751.000000
mean,0.542255,0.076948,0.592436,0.068142,0.038990,0.212112,0.686653,0.274713,0.455133,0.749517,...,0.026926,0.014663,0.013863,0.021861,0.015196,0.016796,0.012263,0.011730,0.020261,0.011197
std,0.498278,0.079989,0.105860,0.078414,0.115885,0.102592,0.078702,0.090017,0.162731,0.071702,...,0.161889,0.120215,0.116938,0.146249,0.122348,0.128522,0.110074,0.107683,0.140911,0.105236
min,0.000000,0.000000,0.282128,0.000000,0.000000,0.002630,0.137873,0.006130,0.000000,0.275590,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.033300,0.517811,0.000000,0.000000,0.138118,0.625627,0.207374,0.378062,0.707339,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.066700,0.585989,0.050000,0.000000,0.190926,0.674037,0.277845,0.499942,0.738961,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.100000,0.668395,0.100000,0.000000,0.261726,0.740663,0.335816,0.569962,0.788177,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,0.964381,0.950000,1.000000,1.000000,0.994735,0.790831,0.989870,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
test.describe()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
count,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000,...,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000,2501.000000
mean,0.078341,0.595248,0.070352,0.035086,0.216895,0.686759,0.272280,0.445657,0.747517,0.269768,...,0.025190,0.015594,0.015594,0.020792,0.009996,0.013994,0.010796,0.009196,0.017193,0.010396
std,0.083113,0.107084,0.078087,0.105563,0.104144,0.080892,0.093832,0.167624,0.073756,0.100849,...,0.156733,0.123922,0.123922,0.142715,0.099499,0.117491,0.103361,0.095475,0.130016,0.101449
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067800,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.033300,0.520863,0.000000,0.000000,0.144178,0.625556,0.201004,0.357777,0.705848,0.190813,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.066700,0.589369,0.050000,0.000000,0.196344,0.672800,0.277606,0.493782,0.735714,0.282347,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.100000,0.675038,0.100000,0.000000,0.265710,0.741591,0.332879,0.564543,0.788615,0.344885,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.966667,1.000000,1.000000,1.000000,0.922371,1.000000,1.000000,1.000000,0.974132,0.993140,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
from azureml.core import Dataset
from azureml.data.dataset_factory import DataType

# create a TabularDataset from a delimited file behind a public web url and convert column "Survived" to boolean
web_path_train ='https://raw.githubusercontent.com/AmDeep/Capstone_Project3_AzureML_Nanodegree/main/Data/train.csv'
web_path_test = 'https://raw.githubusercontent.com/AmDeep/Capstone_Project3_AzureML_Nanodegree/main/Data/test.csv'
train_ds = Dataset.Tabular.from_delimited_files(path=web_path_train, set_column_types={'Activity': DataType.to_bool()})
test_ds = Dataset.Tabular.from_delimited_files(path=web_path_test)


In [21]:
train_ds

{
  "source": [
    "https://raw.githubusercontent.com/AmDeep/Capstone_Project3_AzureML_Nanodegree/main/Data/train.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes",
    "SetColumnTypes"
  ]
}

In [22]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='Activity',
    n_cross_validations=5,
    compute_target=compute_target,
    iterations=100,
    max_concurrent_iterations=8)

In [23]:
automl_run=exp.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on AUTOM-ML-cluster with default configuration
Running on remote compute: AUTOM-ML-cluster
Parent Run ID: AutoML_454a03fa-5100-4810-9148-3fd0689336c9

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:  

In [24]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [25]:
automl_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+---------------------------------+---------------------------------+--------------------------------

{'runId': 'AutoML_454a03fa-5100-4810-9148-3fd0689336c9',
 'target': 'AUTOM-ML-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-12-13T04:17:32.212818Z',
 'endTimeUtc': '2020-12-13T05:04:00.320735Z',
 'properties': {'num_iterations': '100',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'AUTOM-ML-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"AUTO_ML","subscription_id":"2552278b-2817-43a7-820e-5a5a53ff9e19","resource_group":"aml-quickstarts-130297","workspace_name":"quick-starts-ws-130297","region":"southcentralus","compute_target":"AUTOM-ML-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":100,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":

In [56]:
best_run,model = automl_run.get_output()


In [27]:
#Exporting most optimal model obtained from AutoML using joblib
import joblib
joblib.dump(model,'optimal_model.joblib')

['optimal_model.joblib']

In [28]:
#Printing the best estimator for prediction from AutoML
print(model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('22',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='goss',
                                                                               class_weight=None,
                                                                               colsample_bytree=0.3966666666666666,
                                                                               importance_type='split',
                                                                               learning_rate=0.05263631578947369,
                                                   

In [29]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
AUTO_ML,AutoML_454a03fa-5100-4810-9148-3fd0689336c9_38,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [58]:
best_run.id

'AutoML_454a03fa-5100-4810-9148-3fd0689336c9_38'

In [59]:
model._final_estimator

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('22',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='goss',
                                                                               class_weight=None,
                                                                               colsample_bytree=0.3966666666666666,
                                                                               importance_type='split',
                                                                               learning_rate=0.05263631578947369,
                                                   

In [57]:
from azureml.core.model import Model
model=Model.register(workspace=ws,model_path="optimal_model1.joblib",model_name="model_automl")

ERROR:azureml.core.model:Error, provided model path "optimal_model1.joblib" cannot be found



WebserviceException: WebserviceException:
	Message: Error, provided model path "optimal_model1.joblib" cannot be found
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error, provided model path \"optimal_model1.joblib\" cannot be found"
    }
}

In [33]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-130297', subscription_id='2552278b-2817-43a7-820e-5a5a53ff9e19', resource_group='aml-quickstarts-130297'), name=model_automl, id=model_automl:1, version=1, tags={}, properties={})

In [35]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

service_name = 'Biological_Activity_Response_Model'

inference_config = InferenceConfig(entry_script='score.py', environment=best_run.get_environment())
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,enable_app_insights=True)

In [54]:
model = Model.register(model_path="outputs",model_name="optimal_model",workspace=ws)

ValueError: max() arg is an empty sequence

In [60]:
from azureml.core.webservice import LocalWebservice
local_config=LocalWebservice.deploy_configuration(port=9000)
local_service=Model.deploy(ws,"test",[model],inference_config,local_config)
local_service.wait_for_deployment(show_output=True)

NotImplementedError: Models must either be of type azureml.core.model.Model or a str path to a file or folder.

In [47]:
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

WebserviceException: WebserviceException:
	Message: Error, provided service name is invalid. It must only consist of lowercase letters, numbers, or dashes, start with a letter, end with a letter or number, and be between 3 and 32 characters long.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error, provided service name is invalid. It must only consist of lowercase letters, numbers, or dashes, start with a letter, end with a letter or number, and be between 3 and 32 characters long."
    }
}